In [1]:
# Import General Utilities
import numpy as np
import os
import sys
import random
from tqdm.notebook import tqdm, trange
import time
import datetime

#Import tensorflow and its requirements
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.keras.datasets as tfds
import tensorflow.experimental.numpy as tnp
from tensorflow.data.experimental import cardinality as dlen
from tensorflow.keras import Model, Input
from tensorflow.keras import layers
AUTOTUNE = tf.data.AUTOTUNE
tnp.experimental_enable_numpy_behavior()

# Import plotting tools and image converters
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL
import imageio
from IPython import display

# Import file path variable handling
from pathlib import Path
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

# Constants
batch_size = 20
buffer_size = 64
epochs = 350
img_height = 256
img_width = 256
num_examples_to_generate = 8

print(tf.__version__)

2.7.0-dev20210803


In [2]:
# Download OASIS Dataset from the provided link

def download_and_unzip(url, extract_to='.'): # Credit to Antoine Hebert
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)
    http_response.close()

root_dir = '/root/.keras/datasets'
data_dir = Path(root_dir) / 'keras_png_slices_data'

try:
    assert(len(os.listdir(data_dir)) == 6)
except:
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)
    _URL = "https://cloudstor.aarnet.edu.au/plus/s/tByzSZzvvVh0hZA/download"
    download_and_unzip(_URL, extract_to=root_dir)

print(f"Data Directory is {data_dir}")
print(f"Contents: \n\r {os.listdir(data_dir)}")

# Folder Directory Paths
train_dir = data_dir / 'keras_png_slices_train'
train_ans_dir = data_dir / 'keras_png_slices_seg_train'
test_dir = data_dir / 'keras_png_slices_test'
test_ans_dir = data_dir / 'keras_png_slices_seg_test'
val_dir = data_dir / 'keras_png_slices_validate'
val_ans_dir = data_dir / 'keras_png_slices_seg_validate'

train_ds_list = tf.data.Dataset.list_files(str(train_dir/'*.*'), shuffle=True)
test_ds_list = tf.data.Dataset.list_files(str(test_dir/'*.*'), shuffle=True)
val_ds_list = tf.data.Dataset.list_files(str(val_dir/'*.*'), shuffle=False)

train_ds = train_ds_list.take(dlen(train_ds_list))
test_ds = test_ds_list.take(dlen(test_ds_list))
val_ds = val_ds_list.take(dlen(val_ds_list))

print(f"Train Set is {dlen(train_ds_list)} images, Test Set is {dlen(test_ds_list)} images and Validation Set is {dlen(val_ds_list)} images.")

Data Directory is \root\.keras\datasets\keras_png_slices_data
Contents: 
 ['keras_png_slices_seg_test', 'keras_png_slices_seg_train', 'keras_png_slices_seg_validate', 'keras_png_slices_test', 'keras_png_slices_train', 'keras_png_slices_validate']
Train Set is 9664 images, Test Set is 544 images and Validation Set is 1120 images.


In [3]:
def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_png(img, channels=1)
    tfimg = tf.image.resize(img, [img_height, img_width])
    tfimg = (tfimg / 255.0) - 0.5 # Normalise data
    # Resize the image to the desired size
    return tfimg

def process_path(file_path):
    #label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img

def configure_for_performance(ds, batches):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=buffer_size)
    ds = ds.batch(batches)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds_raw = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds_raw = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)
#val_ds_raw = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

train_ds = configure_for_performance(train_ds_raw, batch_size)
test_ds = configure_for_performance(test_ds_raw, batch_size)
#val_ds = configure_for_performance(val_ds_raw, batch_size)

In [15]:
class VectorQuantizer(layers.Layer):
    def __init__(self, num_embeddings, embedding_dim, label="vectorizer", beta=0.25, **kwargs):
        super().__init__(**kwargs)
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.label = label
        self.beta = beta  # This parameter is best kept between [0.25, 2] as per the paper.

        # Initialize the embeddings which we will quantize.
        w_init = tf.random_uniform_initializer()
        self.embeddings = tf.Variable(
            initial_value=w_init(
                shape=(self.embedding_dim, self.num_embeddings), dtype="float32"
            ),
            trainable=True,
            name=self.label,
        )

    def call(self, x):
        input_shape = tf.shape(x)

        # Quantization.
        encoding_indices = self.get_code_indices(x)
        encodings = tf.one_hot(encoding_indices, self.num_embeddings)
        quantized = tf.matmul(encodings, self.embeddings, transpose_b=True)
        quantized = tf.reshape(quantized, input_shape)

        # Calculate vector quantization loss and add that to the layer. You can learn more
        # about adding losses to different layers here:
        # https://keras.io/guides/making_new_layers_and_models_via_subclassing/. Check
        # the original paper to get a handle on the formulation of the loss function.
        commitment_loss = self.beta * tf.reduce_mean(
            (tf.stop_gradient(quantized) - x) ** 2
        )
        codebook_loss = tf.reduce_mean((quantized - tf.stop_gradient(x)) ** 2)
        self.add_loss(commitment_loss + codebook_loss)

        # Straight-through estimator.
        quantized = x + tf.stop_gradient(quantized - x)
        return quantized

    def get_code_indices(self, x):
        # Calculate L2-normalized distance between the inputs and the codes.

        # flatten the inputs keeping `embedding_dim` intact.
        input_shape = tf.shape(x)
        flattened_inputs = tf.reshape(x, [-1, self.embedding_dim])
        print(f"Vector Quantizer {self.label} has input shape {input_shape} and flattened shape {flattened_inputs.shape}")
        print(f"Num Embeddings = {self.embedding_dim}, {self.embeddings.shape}")

        similarity = tf.matmul(flattened_inputs, self.embeddings)
        distances = (
            tf.reduce_sum(flattened_inputs ** 2, axis=1, keepdims=True)
            + tf.reduce_sum(self.embeddings ** 2, axis=0)
            - 2 * similarity
        )

        # Derive the indices for minimum distances.
        encoding_indices = tf.argmin(distances, axis=1)
        return encoding_indices


class SkipConn(layers.Layer):
    def __init__(self, in_channels, channels, **kwargs):
        super().__init__(**kwargs)

        self.conv1 = layers.Conv2D(channels, 3, activation="relu", padding="same")
        self.conv2 = layers.Conv2D(in_channels, 1, activation="relu", padding="same")

    def call(self, input):
        x = self.conv1(input)
        x = self.conv2(x)
        out = x + input
        return out


def get_encoder(input_shape, channels=128, strides=4, name='encoder'):
    encoder_inputs = Input(shape=input_shape, name=name+'input')

    if strides == 4:
        x = layers.Conv2D(channels // 2, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
        x = layers.Conv2D(channels, 3, activation="relu", strides=2, padding="same")(x)

    elif strides == 2:
        x = layers.Conv2D(channels // 2, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
        
    x = layers.Conv2D(channels, 3, activation="relu", padding="same")(x)

    for i in range(2):
        x = SkipConn(channels, 32)(x)
        
    return Model(inputs=encoder_inputs, outputs=x, name=name)


def get_decoder(input_shape, out_channels=128, channels=128, strides=4, name='decoder'):
    latent_inputs = Input(shape=input_shape, name=name+'decoder_input')

    x = layers.Conv2D(channels, 3, activation="relu", padding="same")(latent_inputs)

    for i in range(2):
        x = SkipConn(channels, 32)(x)

    if strides == 4:
        x = layers.Conv2DTranspose(channels // 2, 3, activation="relu", strides=2, padding="same")(x)

    decoder_outputs = layers.Conv2DTranspose(out_channels, 3, strides=2, padding="same")(x)
    return Model(inputs=latent_inputs, outputs=decoder_outputs, name=name)


class VQVAE(Model):
    def __init__(self, latent_dim=16, num_embeddings=128, embed_dim=192, n_embed=256):
        super(VQVAE, self).__init__()

        #self.encoder = get_encoder(input_shape=(256, 256, 1), channels=num_embeddings, strides=4)
        self.enc_b = get_encoder(input_shape=(256, 256, 1), channels=latent_dim, strides=4, name="encoder_b")
        self.enc_t = get_encoder(input_shape=(64, 64, latent_dim), channels=latent_dim, strides=2, name="encoder_t")
        #self.quantize_conv_t = layers.Conv2D(embed_dim, 1, activation="relu", padding="same")
        self.quantize_t = VectorQuantizer(num_embeddings, latent_dim, label="t_vq")

        self.dec_t = get_decoder(input_shape=(32, 32, latent_dim), out_channels=latent_dim, channels=latent_dim, strides=2, name="decoder_t")
        self.quantize_b = VectorQuantizer(embed_dim, n_embed, label="b_vq")
        self.upsample_t = layers.Conv2DTranspose(embed_dim, 3, activation="relu", strides=2, padding="same")

        self.dec = get_decoder(input_shape=(64, 64, 2 * latent_dim + embed_dim), out_channels=1, channels=latent_dim, strides=4, name="decoder_b")

    def encode(self, input):
        enc_b = self.enc_b(input)
        enc_t = self.enc_t(enc_b)

        quant_t = self.quantize_t(enc_t)

        dec_t = self.dec_t(quant_t)
        enc_b = tf.concat([dec_t, enc_b], 3)

        quant_b = self.quantize_b(enc_b)
        #upsample_t = self.upsample_t(quant_t)
        #quant = tf.concat([upsample_t, quant_b], 3)
        
        return quant_t, quant_b

    def decode(self, quant_t, quant_b):
        upsample_t = self.upsample_t(quant_t)
        quant = tf.concat([upsample_t, quant_b], 3)
        dec = self.dec(quant)

        return dec

    def encode_to_t(self, input):
        enc_b = self.enc_b(input)
        enc_t = self.enc_t(enc_b)
        return enc_t

    def encode_to_b(self, input):
        enc_b = self.enc_b(input)
        enc_t = self.enc_t(enc_b)

        quant_t = self.quantize_t(enc_t)

        dec_t = self.dec_t(quant_t)
        enc_b = tf.concat([dec_t, enc_b], 3)
        return enc_b

    def call(self, input):
        quant_t, quant_b = self.encode(input)
        dec = self.decode(quant_t, quant_b)

        return dec


In [7]:
def make_vqvae(latent_dim=16, num_embeddings=128, embed_dim=64, n_embed=256):

    enc_b = get_encoder(input_shape=(256, 256, 1), channels=num_embeddings, strides=4, name="encoder_b")
    enc_t = get_encoder(input_shape=(64, 64, num_embeddings), channels=num_embeddings, strides=2, name="encoder_t")
    quantize_t = VectorQuantizer(num_embeddings, latent_dim, label="t_vq")

    dec_t = get_decoder(input_shape=(32, 32, num_embeddings), out_channels=num_embeddings, channels=num_embeddings, strides=2, name="decoder_t")
    quantize_b = VectorQuantizer(n_embed, embed_dim, label="b_vq")
    upsample_t = layers.Conv2DTranspose(embed_dim, 3, activation="relu", strides=2, padding="same")

    dec = get_decoder(input_shape=(64, 64, 2 * num_embeddings + embed_dim), out_channels=1, channels=num_embeddings, strides=4, name="decoder_b")

    inputs = Input(shape=(256, 256, 1))
    encoded_b = enc_b(inputs)
    encoded_t = enc_t(encoded_b)

    quant_t = quantize_t(encoded_t)
    decoded_t = dec_t(quant_t)
    encoded_b = tf.concat([decoded_t, encoded_b], 3)

    quant_b = quantize_b(encoded_b)

    upsampled_t = upsample_t(quant_t)
    quant = tf.concat([upsampled_t, quant_b], 3)
    decoded = dec(quant)

    return Model(inputs, decoded, name="layer2_vqvae")
    
make_vqvae(latent_dim=8, num_embeddings=96).summary()

Model: "layer2_vqvae"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 1  0           []                               
                                )]                                                                
__________________________________________________________________________________________________
encoder_b (Functional)          (None, 64, 64, 96)   186784      ['input_2[0][0]']                
__________________________________________________________________________________________________
encoder_t (Functional)          (None, 32, 32, 96)   144784      ['encoder_b[0][0]']              
__________________________________________________________________________________________________
vector_quantizer_2 (VectorQuan  (None, 32, 32, 96)   768         ['encoder_t[0][0]']   

In [16]:
trained_vqvae_model = VQVAE(latent_dim=128, num_embeddings=256, embed_dim=128)
trained_vqvae_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4))

#trained_vqvae_model.load_weights(f'/content/drive/MyDrive/COMP3710/PatternFlow/recognition/VQVAE_oasis3/trained_models/96filter_VQVAE')
trained_vqvae_model.load_weights(f'/root/trained_model/model020')
#trained_vqvae_model = tf.keras.models.load_model(f'/content/drive/MyDrive/COMP3710/PatternFlow/recognition/VQVAE_oasis3/trained_models/96filter_VQVAE')
#trained_vqvae_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4))
trained_vqvae_model.build((None, 256, 256, 1))
trained_vqvae_model.summary()

ValueError: ignored

In [ ]:
def show_subplot(original, reconstructed):

    original = original + 0.5
    reconstructed = reconstructed + 0.5

    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(original[:, :, 0])
    plt.title("Original")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(reconstructed[:, :, 0])
    plt.title("Reconstructed")
    plt.axis("off")

    plt.show()

def show_generated_image(samples_t, samples_b):

    generated = trained_vqvae_model.decode(samples_t, samples_b)

    for i in range(samples_t.shape[0]):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(samples_t[i][:, :, 0] + 0.5)
        plt.title("Low Detail Encoding")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.imshow(samples_b[i][:, :, 0] + 0.5)
        plt.title("High Detail Encoding")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.imshow(generated[i][:, :, 0] + 0.5)
        plt.title("Original")
        plt.axis("off")

        plt.show()


test_images = next(iter(test_ds))
reconstructions_test = trained_vqvae_model.predict(test_images)

for i in range(5):
    show_subplot(test_images[i], reconstructions_test[i])

In [ ]:
# Testing encoding visualisation
enc_out_t, enc_out_b = trained_vqvae_model.encode(test_images)
print(f"Low filter encoder is {enc_out_t.shape}, High filter encoder is {enc_out_b.shape}")

for i in range(4):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(test_images[i][:, :, 0] + 0.5)
    plt.title("Original")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(enc_out_t[i][:, :, 0] + 0.5)
    plt.title("Low Detail Encoding")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(enc_out_b[i][:, :, 0] + 0.5)
    plt.title("High Detail Encoding")
    plt.axis("off")
    plt.show()


In [ ]:
ssim_total = 0
count = 0

for batch in tqdm(test_ds):
    reconstructed_comp = trained_vqvae_model.predict(batch)

    for i in range(batch.shape[0]):
        original = tf.math.round((batch[i] + 0.5) * 255.0)
        reconstruction = tf.math.round((reconstructed_comp[i] + 0.5) * 255.0)
        ssim = tf.image.ssim(original, reconstruction, max_val=255.0)
        ssim_total += ssim
        count += 1

print(f"Average SSIM score is {ssim_total / count}.")

In [12]:
# Generate an encoded dataset

def codebooks_t(ds):
    encoded_outputs = trained_vqvae_model.encode_to_t(ds)
    #flat_enc_outputs = tf.reshape(encoded_outputs, [-1, encoded_outputs.shape[-1]])
    codebook_indices = trained_vqvae_model.quantize_t.get_code_indices(encoded_outputs)#flat_enc_outputs)
    codebook_indices = codebook_indices.numpy().reshape(encoded_outputs.shape[:-1])

    return codebook_indices

def codebooks_b(ds):
    encoded_outputs = trained_vqvae_model.encode_to_b(ds)
    #flat_enc_outputs = tf.reshape(encoded_outputs, [-1, encoded_outputs.shape[-1]])
    codebook_indices = trained_vqvae_model.quantize_b.get_code_indices(encoded_outputs)#flat_enc_outputs)
    print(codebook_indices.shape)
    codebook_indices = codebook_indices.numpy().reshape(encoded_outputs.shape[:-1])

    return codebook_indices

pixel_ds = configure_for_performance(train_ds_raw, 10)

codebook_tensor_t = 0
codebook_tensor_b = 0
first = True

for batch in tqdm(pixel_ds, desc=f"Extracting Codebook Indices:"):
    if first:
        codebook_tensor_t = codebooks_t(batch)
        codebook_tensor_b = codebooks_b(batch)
        first = False
    else:
        codebook_tensor_t = tf.concat([codebook_tensor_t, codebooks_t(batch)], 0)
        codebook_tensor_b = tf.concat([codebook_tensor_b, codebooks_b(batch)], 0)
        
print(codebook_tensor_b.shape)

Extracting Codebook Indices::   0%|          | 0/967 [00:00<?, ?it/s]

Vector Quantizer t_vq has input shape [ 10  32  32 128] and flattened shape (10240, 128)
Vector Quantizer t_vq has input shape [ 10  32  32 128] and flattened shape (10240, 128)
Vector Quantizer b_vq has input shape [ 10  64  64 256] and flattened shape (81920, 128)
(81920,)


ValueError: ignored

In [ ]:
# PixelCNN implementation
# The first layer is the PixelCNN layer. This layer simply
# builds on the 2D convolutional layer, but includes masking.
class PixelConvLayer(layers.Layer):
    def __init__(self, mask_type, **kwargs):
        super(PixelConvLayer, self).__init__()
        self.mask_type = mask_type
        self.conv = layers.Conv2D(**kwargs)

    def build(self, input_shape):
        # Build the conv2d layer to initialize kernel variables
        self.conv.build(input_shape)
        # Use the initialized kernel to create the mask
        kernel_shape = self.conv.kernel.get_shape()
        self.mask = np.zeros(shape=kernel_shape)
        self.mask[: kernel_shape[0] // 2, ...] = 1.0
        self.mask[kernel_shape[0] // 2, : kernel_shape[1] // 2, ...] = 1.0
        if self.mask_type == "B":
            self.mask[kernel_shape[0] // 2, kernel_shape[1] // 2, ...] = 1.0
        self.mask = tf.cast(self.mask, tf.float32)

    def call(self, inputs):
        self.conv.kernel.assign(self.conv.kernel * self.mask)
        return tf.cast(self.conv(inputs), tf.float32)


# Next, we build our residual block layer.
# This is just a normal residual block, but based on the PixelConvLayer.
class ResidualBlock(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(
            filters=filters, kernel_size=1, activation="relu"
        )
        self.pixel_conv = PixelConvLayer(
            mask_type="B",
            filters=filters // 2,
            kernel_size=3,
            activation="relu",
            padding="same",
        )
        self.conv2 = layers.Conv2D(
            filters=filters, kernel_size=1, activation="relu"
        )

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pixel_conv(x)
        x = self.conv2(x)
        return layers.add([inputs, x])


def pixel_cnn(shape, filters=128, kernels=64, embeddings=128, name="pixel_cnn"):
    pixelcnn_inputs = Input(shape=shape, dtype=tf.int32)
    ohe = tf.one_hot(pixelcnn_inputs, embeddings)
    ohe = tf.cast(ohe, tf.float32)
    x = PixelConvLayer(
        mask_type="A", filters=filters, kernel_size=kernels, activation="relu", padding="same"
    )(ohe)

    for _ in range(2):
        x = ResidualBlock(filters=filters)(x)

    for _ in range(2):
        x = PixelConvLayer(
            mask_type="B",
            filters=filters,
            kernel_size=1,
            strides=1,
            activation="relu",
            padding="valid",
        )(x)

    out = layers.Conv2D(
        filters=embeddings, kernel_size=1, strides=1, padding="valid"
    )(x)

    return Model(pixelcnn_inputs, out, name=name)

In [ ]:
pixel_cnn_t = pixel_cnn((32, 32), filters=128, kernels=64, embeddings=128, name="pixel_cnn_top")
pixel_cnn_t.load_weights(f"/root/trained_pixel_model/model019_t")
pixel_cnn_b = pixel_cnn((64, 64), filters=128, kernels=64, embeddings=256, name="pixel_cnn_bottom")
pixel_cnn_b.load_weights(f"/root/trained_pixel_model/model019_b")

In [ ]:
# Create a mini sampler model
def gen_priors_layer(model, num_samples=10):
    inputs = Input(shape=model.input_shape[1:])
    x = model(inputs, training=False)
    dist = tfp.distributions.Categorical(logits=x)
    sampled = dist.sample()
    sampler = Model(inputs, sampled)

    # Create an empty array of priors.
    priors = np.zeros(shape=(num_samples,) + (model.input_shape)[1:])
    num_samples, rows, cols = priors.shape

    # Iterate over the priors because generation has to be done sequentially pixel by pixel.
    for row in trange(rows, desc="Predicting priors"):
        for col in range(cols):
            # Feed the whole array and retrieving the pixel value probabilities for the next
            # pixel.
            probs = sampler.predict(priors)
            # Use the probabilities to pick pixel values and append the values to the priors.
            priors[:, row, col] = probs[:, row, col]
    return priors

priors_t = gen_priors_layer(pixel_cnn_t)
priors_b = gen_priors_layer(pixel_cnn_b)

In [ ]:
def gen_encodings(priors, embeddings, pretrained_embeddings, shape):
    # Perform an embedding lookup.
    priors_ohe = tf.one_hot(priors.astype("int32"), embeddings).numpy()
    print(priors_ohe.shape)
    quantized = tf.matmul(priors_ohe.astype("float32"), pretrained_embeddings)
    print(quantized.shape)
    #quantized = tf.reshape(quantized, (-1, *shape))
    return priors_ohe#quantized

pad_t = gen_encodings(priors_t, 96, trained_vqvae_model.quantize_t.embeddings, (32, 32, 96))
pad_b = gen_encodings(priors_b, 192, trained_vqvae_model.quantize_b.embeddings, (64, 64, 192))

show_generated_image(pad_t, pad_b)

In [ ]:
pixel_cnn_t = pixel_cnn((32, 32), filters=128, kernels=64, embeddings=128, name="pixel_cnn_top")

pixel_cnn_t.compile(
    optimizer=tf.keras.optimizers.Adam(3e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

pixel_cnn_t.fit(
    x=codebook_tensor_t,
    y=codebook_tensor_t,
    batch_size=128,
    epochs=30,
    validation_split=0.1,
)


In [ ]:
os.makedirs(f"/root/trained_pixel_model/model019_t")
pixel_cnn_t.save(f"/root/trained_pixel_model/model019_t")

In [ ]:
pixel_cnn_b = pixel_cnn((64, 64), filters=128, kernels=64, embeddings=256, name="pixel_cnn_bottom")

pixel_cnn_b.compile(
    optimizer=tf.keras.optimizers.Adam(3e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

pixel_cnn_b.fit(
    x=codebook_tensor_b,
    y=codebook_tensor_b,
    batch_size=20,
    epochs=2,
    validation_split=0.1,
)

In [ ]:
#os.makedirs(f"/root/trained_pixel_model/model019_b")
pixel_cnn_b.save(f"/root/trained_pixel_model/model019_b")